<a href="https://colab.research.google.com/github/MEHULSHARMA26/Data_analysis_Learn/blob/main/EDA_online_retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
!wget https://archive.ics.uci.edu/static/public/352/online+retail.zip

--2025-03-04 06:01:21--  https://archive.ics.uci.edu/static/public/352/online+retail.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘online+retail.zip.1’

online+retail.zip.1     [    <=>             ]  22.62M  31.5MB/s    in 0.7s    

2025-03-04 06:01:22 (31.5 MB/s) - ‘online+retail.zip.1’ saved [23715478]



In [ ]:
!unzip online+retail.zip

Archive:  online+retail.zip
replace Online Retail.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df1 = pd.read_excel("Online Retail.xlsx",dtype={'InvoiceNo':'string','StockCode':'string','Description':'string','Country':'string'})
df1.head(1)

In [ ]:

df1.shape
df1.isnull().sum()

In [ ]:
df1.info()

### Data Cleaning: Handle Missing Values

In [ ]:
df1[df1.Description.isnull()]

df1[df1.Description.isnull()] => it displays all the info about the records where description is null

In [ ]:
df1[df1.StockCode=='22139']

In [ ]:
df1[df1.StockCode=='22139'].Description.mode()

In [ ]:
most_frq = df1[['StockCode','Description']].value_counts().reset_index()
most_frq

In [ ]:
most_frq[most_frq.StockCode=='85123A']

In [ ]:
most_frq= most_frq.groupby('StockCode').head(1)
most_frq

In [ ]:
most_frq[most_frq.StockCode=='85123A']

In [ ]:
most_frq.columns =  ['StockCode','freq_Description','count']

most_frq

In [ ]:
df2 = df1.merge(most_frq,on='StockCode',how='left')
df2

In [ ]:
df2['Description'] = df2['freq_Description']
df2

In [ ]:
 df2.isnull().sum()

In [ ]:
df2.dropna(subset=['Description'],inplace=True)
df2.isnull().sum()
df2

In [ ]:
df2.drop(columns=["freq_Description","count"],inplace=True)
df2.head()

In [ ]:
 df2.describe()

In [ ]:
df2[df2.Quantity<0]

In [ ]:
df3 = df2[(df2.UnitPrice > 0) & (df2.Quantity > 0)]
df3.describe()


In [ ]:
df3.Quantity.quantile(0.99)

In [ ]:
df3[df3.Quantity > 1500]

### Feature Engineering: Create New Columns

In [ ]:
# prompt: generate me column for Total sales using Quantity and unit price  and store it in new dataframe as df4

df4 = df3.copy()
df4['TotalSales'] = df4['Quantity'] * df4['UnitPrice']
df4.head(3)


In [ ]:
df4.info()

In [ ]:
 df4["Month"] = df4['InvoiceDate'].dt.month
 df4.sample(5)

### Visualization and EDA

### Plot monthly sales

In [ ]:
monthly_sales = df4.groupby('Month')['TotalSales'].sum()
monthly_sales.plot(kind = "line", title = "Monthly sales",marker ='o')
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.grid()
plt.show()

***Insights***

Total sales started rising sharply after august and reached to it's peak in november.this is highly likely due to the festive season at the end of the year

2. Top 5 countries (By sales)

In [ ]:
# prompt: plot a horizontal bar chart for top 5 countries by total sales

import matplotlib.pyplot as plt
top_countries = df4.groupby('Country')['TotalSales'].sum().sort_values(ascending=False).head(5)
plt.figure(figsize=(10, 6))
plt.barh(top_countries.index, top_countries.values)
plt.xlabel('Total Sales')
plt.ylabel('Country')
plt.title('Top 5 Countries by Total Sales')
plt.show()


In [ ]:
# prompt: plot same chart as above but this time use percentage contribution.Show % on the bar

import matplotlib.pyplot as plt

# Calculate percentage contribution

country_wise_sales = df4.groupby('Country')['TotalSales'].sum()
total_sales = country_wise_sales.sum()

top_5_countries = country_wise_sales.sort_values(ascending=False).head(5)
percentage_contributions = (top_5_countries / total_sales) * 100


# Create the bar chart with percentage labels
plt.figure(figsize=(10, 6))
bars = plt.barh(top_countries.index, percentage_contributions)
plt.xlabel('Percentage Contribution to Total Sales')
plt.ylabel('Country')
plt.title('Top 5 Countries by Percentage Contribution to Total Sales')

# Add percentage labels on the bars
for bar, percentage in zip(bars, percentage_contributions):
    plt.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, f'{percentage:.1f}%',
             va='center', ha='left', fontsize=10, color='black')

plt.show()


In [ ]:
product_wise_sales = df4.groupby('StockCode')['TotalSales'].sum()

top_5_products = product_wise_sales.sort_values(ascending=False).head(5)
top_5_products.plot(kind='barh',color = 'skyblue')
plt.title('Product-wise Sales')
plt.xlabel('Total Sales')
plt.ylabel('Product Stock Code')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
for stock_code in top_5_products.index:
    description = df4[df4['StockCode'] == stock_code]['Description'].iloc[0]
    print(f"Stock Code: {stock_code}, Description: {description}")

In [ ]:
# prompt: plot same chart as above for product sales but use percentage this time.Show % on the bar

import matplotlib.pyplot as plt

# Calculate percentage contribution

total_sales = product_wise_sales.sum()
percentages = (top_5_products / total_sales) * 100


# Create the bar chart with percentage labels
plt.figure(figsize=(10, 6))
bars = plt.barh(top_5_products.index, percentage_contributions)
plt.xlabel('Percentage Contribution to Total Sales')
plt.ylabel('Product Stock Code')
plt.title('Top 5 Products by Percentage Contribution to Total Sales')
plt.grid(axis = 'x')

# Add percentage labels on the bars
for bar, percentage in zip(bars, percentages):
    plt.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, f'{percentage:.1f}%',
             va='center')


plt.show()




### RFM ANALYSIS

recency,frequency,monetary analysis

In [ ]:
current_dt = df4["InvoiceDate"].max() + pd.Timedelta(days = 1)
current_dt

In [ ]:
rfm = df4.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (current_dt - x.max()).days,
    "InvoiceNo": 'count',
    "TotalSales": 'sum'
})

rfm.columns = ['Recency','Frequency','Monetary']
rfm.head(5)

### Segment Customer based on RFM

In [ ]:
rfm['R_Segment'] = pd.qcut(rfm['Recency'],4,labels=[4,3,2,1])
rfm['F_Segment'] = pd.qcut(rfm['Frequency'],4,labels=[1,2,3,4])
rfm['M_Segment'] = pd.qcut(rfm['Monetary'],4,labels=[1,2,3,4])
rfm['RFM_Score'] = rfm[['R_Segment','F_Segment','M_Segment']].sum(axis=1)
rfm

In [ ]:
rfm.sort_values("RFM_Score",ascending=False)

### 5 Customer Churn analysis

In [ ]:
#Create a basket matrix for association rule mining
customer_last_purchase = df4.groupby("CustomerID")['InvoiceDate'].max()
customer_last_purchase.head(5)

In [ ]:
 current_dt

In [ ]:
customer_last_purchase = (current_dt - customer_last_purchase).dt.days
customer_last_purchase.head(5)

In [ ]:
#Define churn threshold (e.g., 90 days without purchase)
churn_threshold = 90
churned_customers = customer_last_purchase[customer_last_purchase > churn_threshold]
churned_customers.head(5)